In [ ]:
# Exercise 1
# In a senario where there are more variables than observations, using the least squares method would not 
# be ideal because of the tendancy to have a low bias, but high variance and not having a unique solution.
# In this case, the best method to use is ridge regression because however it will have a high bias, the 
# variance will be much lower than least squares.

In [ ]:
# Exercise 2
# (f)

In [ ]:
# Exercise 3
# (d)

In [ ]:
# Exercise 4
# From the error in the model being high, this means there is high variance and increasing the vaariable
# lambda should help with the model being more accurate and having less variance in the results. 

In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV

s3 = boto3.resource('s3')
bucket_name = 'data-445-timlincoln'
bucket = s3.Bucket(bucket_name)

file_key = 'CarPrice_Assignment.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the data-file
car_price = pd.read_csv(file_content_stream)
car_price.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [3]:
# Defining input and target variables
X = car_price[['wheelbase', 'enginesize', 'horsepower', 'compressionratio', 'peakrpm', 'citympg', 'highwaympg']]
Y = car_price['price']

# Defining the list
coef = list()

for i in range(0, 1000):
    
    # Splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    # Running LASSO cross-validation
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)
    
    # Building lasso
    lasso_md = Lasso(alpha = lasso_cv.alpha_, normalize = True).fit(X_train, Y_train)
    
    # Storing coeficients
    coef.append(lasso_md.coef_)
    
# Putting list as data-frame
coef_data = pd.DataFrame(coef)

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing

In [4]:
coef_data.head()

,0,1,2,3,4,5,6
0,194.285507,111.256178,44.202717,315.245573,2.075172,-143.049098,-0.0
1,172.856957,113.402209,41.288611,319.708077,2.611297,-144.112148,-0.0
2,194.163511,102.224831,49.484507,292.666136,1.886904,-124.431378,0.0
3,204.244291,102.685330,56.106910,328.291898,2.220669,-94.046174,-0.0
4,190.188602,109.139305,49.137677,302.386867,2.600736,-100.401615,-0.0


In [5]:
sum(coef_data[6] == 0.0)

742

In [6]:
# Defining input and target variables
X = car_price[['wheelbase', 'enginesize', 'horsepower', 'compressionratio', 'peakrpm', 'citympg']]
Y = car_price['price']

# Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

# Defining L2
def l2_normalization(X):
    
    x_mean = np.mean(X)
    l2 = np.sqrt(sum(X**2))
    return (X - x_mean) / 12

X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)

In [7]:
# Linear Regression
lm_md = LinearRegression(). fit(X_train, Y_train)

# Predicting on test
lm_pred = lm_md.predict(X_test)

# Computing the mse of the linear regression
mse1 = np.mean(np.power(Y_test - lm_pred, 2))
print('The mse of the linear model is', mse1)

The mse of the linear model is 8749812.019134426


In [8]:
# Defining the list
lamlist = list()

# Creating loop
for n in range(0, 100):

    # Ridge regression
    ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv = 5).fit(X_train, Y_train)

    # Extracting the best lambda
    cv_lambda = ridge_cv.alpha_
    lamlist.append(cv_lambda)
    
# Putting list as data-frame
lamlist_data = pd.DataFrame(lamlist)
lamlist_data.head()

,0
0,0.001
1,0.001
2,0.001
3,0.001
4,0.001


In [9]:
# Building the ridge model
ridge_md = Ridge(alpha = cv_lambda).fit(X_train, Y_train)

# Predicting on test
ridge_pred = ridge_md.predict(X_test)

# Computing the mse of the ridge regression model
mse2 = np.mean(np.power(Y_test - ridge_pred, 2))
print('The mse of the ridge model is', mse2)

The mse of the ridge model is 8697564.640334943


In [ ]:
# From the results of both models, the model I would use for predicting car prices would be the ridge model
# because the mse was lower than the mse for the linear model. 